In [1]:
from earth_engine import ee_precipitation_data
import csv
params = {
    "station_name": "tumaco",
    "variable_name": "Precipitación acumulada",
    "start_date": "2010-01-01",
    "end_date": "2010-12-31"
}


EEException: Cannot authenticate: Invalid request.

In [ ]:

# serie de earth engine
collection = {
    "collection": "UCSB-CHG/CHIRPS/DAILY"  # Cambia esto por tu colección específica
}
ee_serie = ee_precipitation_data(params, collection)
ee_serie

In [ ]:

# Inicializar Google Earth Engine
ee.Initialize()

# Definir el área de interés: Costa del Pacífico de Colombia
define_coastal_area = ee.Geometry.Polygon([
    [
        [-79.0, 1.0],  # Punto sur (aproximadamente)
        [-79.0, 7.0],  # Punto norte (aproximadamente)
        [-77.0, 7.0],
        [-77.0, 1.0]
    ]
])

# Definir la colección de datos: CHIRPS (Precipitación diaria)
collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

# Fechas de interés
start_date = '2000-01-01'
end_date = '2023-12-31'

# Obtener los puntos de muestreo cercanos a la costa
sampling_points = define_coastal_area.randomPoints(maxPoints=50, seed=1)

# Crear una función para extraer series temporales por punto
def extract_timeseries(point):
    coords = point.geometry().coordinates().getInfo()
    timeseries = collection
    .filterDate(start_date, end_date)
    .select('precipitation')
    .map(lambda image: image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point.geometry(),
        scale=10000
    ).set({'date': image.date().format('YYYY-MM-dd')}))

    timeseries_list = timeseries.getInfo()
    data = [{
        'Longitude': coords[0],
        'Latitude': coords[1],
        'Date': entry['date'],
        'Precipitation': entry['properties']['precipitation']
    } for entry in timeseries_list]

    return data

# Extraer series temporales de cada punto
data_list = []
for i in range(sampling_points.size().getInfo()):
    point = sampling_points.geometry().get(i)
    data_list.extend(extract_timeseries(point))

# Crear un DataFrame de Pandas
precipitation_df = pd.DataFrame(data_list)

# Guardar como CSV
output_file = "precipitation_timeseries_pacific_coast.csv"
precipitation_df.to_csv(output_file, index=False)

print(f"El archivo {output_file} se ha generado correctamente.")
